In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd 

In [3]:
data = pd.read_csv("/content/drive/Shareddrives/다빈치/LPOINT_BIG_COMP/LPOINT_BIG_COMP_02_PDDE.csv")
df = data.copy()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df.head()

,cust,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct
0,M430112881,A01000001113,1,A01,A010039,PD0290,20210101,10,15000.0,1
1,M646853852,A01000002265,1,A01,A010025,PD1369,20210101,10,79700.0,1
2,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1
3,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1
4,M430112881,A01000004946,1,A01,A010039,PD0290,20210101,10,19000.0,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4381743 entries, 0 to 4381742
Data columns (total 10 columns):
 #   Column   Dtype  
---  ------   -----  
 0   cust     object 
 1   rct_no   object 
 2   chnl_dv  int64  
 3   cop_c    object 
 4   br_c     object 
 5   pd_c     object 
 6   de_dt    int64  
 7   de_hr    int64  
 8   buy_am   float64
 9   buy_ct   int64  
dtypes: float64(1), int64(4), object(5)
memory usage: 334.3+ MB


In [6]:
feature_df = pd.DataFrame()
feature_df = df[['cust', 'pd_c', 'de_hr']]
feature_df.head()

,cust,pd_c,de_hr
0,M430112881,PD0290,10
1,M646853852,PD1369,10
2,M430112881,PD0290,10
3,M430112881,PD0290,10
4,M430112881,PD0290,10


In [7]:
# 확인용
# tmp[tmp['cust'] == 'M430112881']

In [8]:
# user_total_orders = 고객이 총 주문한 횟수 = 고객별 영수증 유니크한 수

tmp = df.groupby(['cust'])['rct_no'].nunique()
tmp = pd.DataFrame(tmp)
tmp.reset_index(level=['cust'], inplace = True)

feature_df = pd.merge(feature_df, tmp, how='left', left_on='cust', right_on='cust')
feature_df.rename(columns = {'rct_no' : 'user_total_orders'}, inplace = True)
feature_df

,cust,pd_c,de_hr,user_total_orders
0,M430112881,PD0290,10,102
1,M646853852,PD1369,10,97
2,M430112881,PD0290,10,102
3,M430112881,PD0290,10,102
4,M430112881,PD0290,10,102
...,...,...,...,...
4381738,M533286446,PD0507,23,131
4381739,M533286446,PD0507,23,131
4381740,M533286446,PD0507,23,131
4381741,M533286446,PD0507,23,131


In [9]:
# user_total_items = 고객이 총 주문한 상품 수 = 구매한 수량 (buy_ct) 총합

tmp = df.groupby(df['cust']).sum()
tmp.reset_index(level=['cust'], inplace = True)
tmp = tmp[['cust', 'buy_ct']]

feature_df = pd.merge(feature_df, tmp, how='left', left_on='cust', right_on='cust')
feature_df.rename(columns = {'buy_ct' : 'user_total_items'}, inplace = True)
feature_df

,cust,pd_c,de_hr,user_total_orders,user_total_items
0,M430112881,PD0290,10,102,580
1,M646853852,PD1369,10,97,407
2,M430112881,PD0290,10,102,580
3,M430112881,PD0290,10,102,580
4,M430112881,PD0290,10,102,580
...,...,...,...,...,...
4381738,M533286446,PD0507,23,131,306
4381739,M533286446,PD0507,23,131,306
4381740,M533286446,PD0507,23,131,306
4381741,M533286446,PD0507,23,131,306


In [10]:
# total_distinct_items = 고객이 총 주문한 상품 종류의 개수 = 고객별 상품명 유니크한 수

tmp = df.groupby(['cust'])['pd_c'].nunique()
tmp = pd.DataFrame(tmp)
tmp.reset_index(level=['cust'], inplace = True)

feature_df = pd.merge(feature_df, tmp, how='left', left_on='cust', right_on='cust')
feature_df.rename(columns = {'pd_c_x' : 'pd_c'}, inplace = True)
feature_df.rename(columns = {'pd_c_y' : 'total_distinct_items'}, inplace = True)
feature_df

,cust,pd_c,de_hr,user_total_orders,user_total_items,total_distinct_items
0,M430112881,PD0290,10,102,580,168
1,M646853852,PD1369,10,97,407,147
2,M430112881,PD0290,10,102,580,168
3,M430112881,PD0290,10,102,580,168
4,M430112881,PD0290,10,102,580,168
...,...,...,...,...,...,...
4381738,M533286446,PD0507,23,131,306,74
4381739,M533286446,PD0507,23,131,306,74
4381740,M533286446,PD0507,23,131,306,74
4381741,M533286446,PD0507,23,131,306,74


In [11]:
# user_average_days_between_orders = 고객이 주문까지 걸리는 평균 기간 = ?
# 캐글 데이터셋에는 이전 주문 시간과 현재 주문 시간의 차가 나와 있음 ㅜㅜ

In [12]:
# user_average_basket = 고객의 장바구니 평균 상품개수 
#                     = 고객이 총 주문한 상품 수 / 고객이 총 주문한 횟수

feature_df['user_average_basket'] = feature_df['user_total_items'] / feature_df['user_total_orders']
feature_df

,cust,pd_c,de_hr,user_total_orders,user_total_items,total_distinct_items,user_average_basket
0,M430112881,PD0290,10,102,580,168,5.686275
1,M646853852,PD1369,10,97,407,147,4.195876
2,M430112881,PD0290,10,102,580,168,5.686275
3,M430112881,PD0290,10,102,580,168,5.686275
4,M430112881,PD0290,10,102,580,168,5.686275
...,...,...,...,...,...,...,...
4381738,M533286446,PD0507,23,131,306,74,2.335878
4381739,M533286446,PD0507,23,131,306,74,2.335878
4381740,M533286446,PD0507,23,131,306,74,2.335878
4381741,M533286446,PD0507,23,131,306,74,2.335878


In [13]:
# order_hour_of_day = 고객이 상품을 구매한 시간 = de_hr 이름 변경

# feature_df.rename(columns = {'de_hr' : 'order_hour_of_day'}, inplace = True)
# feature_df

In [14]:
# 평일 구매 비율 & 주말 구매 비율

from datetime import datetime, date

sample_df = df[['cust', 'rct_no', 'de_dt']]
sample_df = sample_df.drop_duplicates()
sample_df['weekday'] = sample_df['de_dt'].astype(str)
sample_df['weekday'] = sample_df['weekday'].str[0:4] + "-" + sample_df['weekday'].str[4:6] + "-" + sample_df['weekday'].str[6:8]
sample_df['weekday'] = pd.to_datetime(sample_df['weekday'])
sample_df['weekday'] = sample_df.weekday.dt.day_name()

day_set = {'weekday' : {'Monday':1, 'Tuesday':1, 'Wednesday':1, 'Thursday':1, 'Friday':1, 'Saturday':0, 'Sunday':0}}
sample_df = sample_df.replace(day_set)

tmp = sample_df.groupby(['cust'])['weekday'].sum()
tmp = pd.DataFrame(tmp)
tmp.reset_index(level=['cust'], inplace = True)

feature_df = pd.merge(feature_df, tmp, how='left', left_on='cust', right_on='cust')
feature_df

,cust,pd_c,de_hr,user_total_orders,user_total_items,total_distinct_items,user_average_basket,weekday
0,M430112881,PD0290,10,102,580,168,5.686275,88
1,M646853852,PD1369,10,97,407,147,4.195876,76
2,M430112881,PD0290,10,102,580,168,5.686275,88
3,M430112881,PD0290,10,102,580,168,5.686275,88
4,M430112881,PD0290,10,102,580,168,5.686275,88
...,...,...,...,...,...,...,...,...
4381738,M533286446,PD0507,23,131,306,74,2.335878,89
4381739,M533286446,PD0507,23,131,306,74,2.335878,89
4381740,M533286446,PD0507,23,131,306,74,2.335878,89
4381741,M533286446,PD0507,23,131,306,74,2.335878,89


In [15]:
feature_df['weekday_rate'] = feature_df['weekday'] / feature_df['user_total_orders']
feature_df['weekend_rate'] = 1 - feature_df['weekday_rate']
feature_df

,cust,pd_c,de_hr,user_total_orders,user_total_items,total_distinct_items,user_average_basket,weekday,weekday_rate,weekend_rate
0,M430112881,PD0290,10,102,580,168,5.686275,88,0.862745,0.137255
1,M646853852,PD1369,10,97,407,147,4.195876,76,0.783505,0.216495
2,M430112881,PD0290,10,102,580,168,5.686275,88,0.862745,0.137255
3,M430112881,PD0290,10,102,580,168,5.686275,88,0.862745,0.137255
4,M430112881,PD0290,10,102,580,168,5.686275,88,0.862745,0.137255
...,...,...,...,...,...,...,...,...,...,...
4381738,M533286446,PD0507,23,131,306,74,2.335878,89,0.679389,0.320611
4381739,M533286446,PD0507,23,131,306,74,2.335878,89,0.679389,0.320611
4381740,M533286446,PD0507,23,131,306,74,2.335878,89,0.679389,0.320611
4381741,M533286446,PD0507,23,131,306,74,2.335878,89,0.679389,0.320611


In [16]:
# user_total_items = 고객별 총 구매 수량
# total_distinct_items = 고객별 구매 상품 종류 수
# user_average_basket = 고객별 영수증 속 평균 상품 수
# weekday_rate = 평일 구매 비율 (평일 영수증 총 수/영수증 총 수)
# weekend_rate = 주말 구매 비율 (주말 영수증 총 수/영수증 총 수)

cols = feature_df.columns[[0, 4, 5, 6, 8, 9]]
feature_df = feature_df[cols]
feature_df = feature_df.drop_duplicates()
feature_df

,cust,user_total_items,total_distinct_items,user_average_basket,weekday_rate,weekend_rate
0,M430112881,580,168,5.686275,0.862745,0.137255
1,M646853852,407,147,4.195876,0.783505,0.216495
8,M669384418,122,62,2.711111,0.755556,0.244444
10,M886923095,123,50,1.808824,0.779412,0.220588
11,M707991383,385,145,1.645299,0.410256,0.589744
...,...,...,...,...,...,...
4381304,M423644302,4,1,4.000000,1.000000,0.000000
4381457,M316481461,1,1,1.000000,1.000000,0.000000
4381508,M765148247,1,1,1.000000,0.000000,1.000000
4381570,M498069014,5,1,5.000000,1.000000,0.000000


In [17]:
feature_df.to_csv('/content/drive/Shareddrives/다빈치/Jdy/feature.csv')

In [18]:
# days_since_prior_order = 고객의 재주문까지 기간
# 기존 캐글에 그대로 데이터셋이 있음 ㅜㅜ
# 고객의 재주문까지 "평균" 기간 

In [19]:
# days_since_ratio = 고객이 재주문까지 기간 / 고객의 주문까지 평균 기간

In [20]:
# feature_df['ID'] = feature_df['cust'] + '_' + feature_df['pd_c']
# cols = feature_df.columns[[-1, 2, 3, 4, 5, 6]]
# feature_df = feature_df[cols]
# feature_df = feature_df.drop_duplicates()
# feature_df